# Работа с текстовыми файлами

Проведем простой анализ повести А. С. Пушкина "Капитанская дочка": посчитаем количество употреблений каждого слова и найдем самые популярные, то есть наиболее часто встречаемые слова.

План действий:
1. чтение текста из файла;
1. составление словаря слов;
1. сортировка словаря слов;
1. удаление стоп-слов;
1. сохранение результатов в файл.

## 1. Чтение из txt-файла и подготовка к составлению словаря слов
Считываем текст из файла в одну строковую переменную, удаляя пробельные символы по краям строки.

In [ ]:
text = ''                          # Создаем пустую строку, в которой будем хранить текст
with open('dochka.txt') as infile: # Если будут проблемы с кодировкой, попробуйте encoding='utf-8'
    for line in infile:            # Последовательно для всех строк в файле:
        text += line.strip() + ' ' # удаляем пробельные символы по краям строки,
                                   # добавляем новую строку и пробел к переменной text 

# Смотрим, как выглядит начало считанного текста (первые 100 символов)
print(text[:100])                  

 Александр Пушкин КАПИТАНСКАЯ ДОЧКА Береги честь смолоду. Пословица.  ГЛАВА I СЕРЖАНТ ГВАРДИИ — Был 


Хотим составить словарь, в котором ключ — уникальное слова текста, а значение — частота, то есть количество таких слов. Сначала нужно получить список всех слов текста.

In [ ]:
almost_words = text.split(' ')  # Разбиваем текст в список по пробелам, 
                                # Получим список "почти-слов" (к ним "приклеены" знаки препинания)
print(almost_words[:10])        # Смотрим на начало списка "почти-слов"

['', 'Александр', 'Пушкин', 'КАПИТАНСКАЯ', 'ДОЧКА', 'Береги', 'честь', 'смолоду.', 'Пословица.', '']


In [ ]:
words = []                                           # Создаем пустой список для хранения слов
for almost_word in almost_words:                     # Последовательно для всех "почти-слов" записываем слово 
                                                     # маленькими буквами и убираем знаки препинания с краев 
    word = almost_word.lower().strip('.,;:!«»?—()"') 
    if word != '':          # Если получился не пустой элемент,
        words.append(word)  # добавляем его к  списку слов

# Смотрим на начало списка слов 
print(words[:10])   

['александр', 'пушкин', 'капитанская', 'дочка', 'береги', 'честь', 'смолоду', 'пословица', 'глава', 'i']


## 2. Составление словаря слов
Теперь создадим словарь по списку слов. Так как нам придется делать это неоднократно, напишем соответствующую функцию.

In [ ]:
# Функция создания словаря по тексту, представленному в виде списка слов
# Входные параметры: текст в виде списка слов
# Возвращаемые значения: словарь {слово: количество употреблений слова}

def create_dict(list_of_words): 
    dictionary = {}                # Создаем пустой словарь
    for word in list_of_words:     # Последовательно для всех слов в списке:
        if word not in dictionary: # если слово встречается впервые,
            dictionary[word] = 1   # указываем, что оно пока одно,
        else:                      # иначе (если слово уже есть в словаре)
            dictionary[word] += 1  # увеличиваем его значение на 1
    return dictionary              # Возвращаем заполненный словарь

Посмотрим на коротком тексте, преобразованном в список слов, как сработает эта функция.

In [ ]:
# Исходный короткий текст: 
# "Сегодня он гуляет, завтра он спит, послезавтра он гуляет."
small_text = 'сегодня он гуляет завтра он спит послезавтра он гуляет'
small_list = small_text.split(' ')
small_dict = create_dict(small_list)
print(small_dict)

{'сегодня': 1, 'он': 3, 'гуляет': 2, 'завтра': 1, 'спит': 1, 'послезавтра': 1}


Функция сработала верно, применим ее к обрабатываемому списку слов.

In [ ]:
dict_of_words = create_dict(words)  # Получаем словарь по заданному списку

Предыдущий маленький словарь мы легко могли посмотреть целиком, а теперь слов слишком много — хочется посмотреть на фрагмент словаря. Напишем соответствующую функцию, которая будет выводить элементы словаря в заданном диапазоне (в том порядке, в котором они хранятся в словаре).

In [ ]:
# Функция вывода элементов словаря в заданном диапазоне
# Входные параметры: словарь, диапазон вывода (номера начала и конца)
# Возвращаемые значения: отсутствуют

def print_dict(dictionary, start, end):   
    i = 0                           # Заводим счетчик
    for w in dictionary:            # Последовательно для всех ключей словаря:
        if i >= start:              # если счетчик не меньше номера начала,
            print(w, dictionary[w]) # печатаем ключ и значение;
        if i == end:                # если счетчик дошел до номера конца,
            break                   # выходим из цикла
        i += 1                      # Увеличиваем счетчик

In [ ]:
print_dict(dict_of_words, 0, 10)    #  Печатаем элементы словаря с 0 по 10 

александр 2
пушкин 1
капитанская 3
дочка 2
береги 2
честь 6
смолоду 2
пословица 2
глава 14
i 1
сержант 2


# 3. Сортировка словаря слов

Сейчас ключи словаря расположены в том порядке, как мы их записывали. А мы хотим узнать самые популярные, то есть наиболее часто употребляемые слова. Но для этого нужно отсортировать словарь по значениям. Встроенной такой функции нет — напишем свою. Сразу оговоримся, что на выходе будет не словарь, а **отсортированный список, состоящий из ключей и значений словаря**.

Сначала по шагам разберем, что будет делать функция. На этапе разработки будем использовать словарь `small_dict`.

In [ ]:
print(small_dict)

{'сегодня': 1, 'он': 3, 'гуляет': 2, 'завтра': 1, 'спит': 1, 'послезавтра': 1}


Преобразуем словарь в список. Для этого нужно преобразовать словарь в пары (ключ, значение) с помощью метода `items()`.

In [ ]:
print(small_dict.items()) # Вспоминаем, как работает метод items()

{'сегодня': 1, 'он': 3, 'гуляет': 2, 'завтра': 1, 'спит': 1, 'послезавтра': 1}
dict_items([('сегодня', 1), ('он', 3), ('гуляет', 2), ('завтра', 1), ('спит', 1), ('послезавтра', 1)])


In [ ]:
list_d = list(small_dict.items()) # Преобразуем словарь в список
print(*list_d, sep = '\n')

('сегодня', 1)
('он', 3)
('гуляет', 2)
('завтра', 1)
('спит', 1)
('послезавтра', 1)


Теперь хотим отсортировать список так, чтобы наверху списка оказались самые популярные слова. Значит, сортировать нужно по частоте, то есть по количеству употреблений слова — по второму элементу.

Но простая сортировка списка отсортирует по первым элементам списка, которые в нашем случае являются строками: 

In [ ]:
sorted_list = sorted(list_d)
print(*sorted_list, sep='\n')

('гуляет', 2)
('завтра', 1)
('он', 3)
('послезавтра', 1)
('сегодня', 1)
('спит', 1)


Если бы мы хотели получить сортировку слов по алфавиту, нас бы это устроило. Но мы хотим найти самые популярные слова. Это значит, что мы хотим отсортировать по "второму столбцу" — второму элементу кортежей. Для этого можем вывести его на первое место, так как сортировка списка начинается с первых элементов кортежей.

Поменяем местами слово и количество употреблений, чтобы то, по чему мы хотим сортировать, стояло на первом месте:

In [ ]:
list_swop = []
for word, count in list_d:          # Последовательно для всех ключей и значений:  
    list_swop.append((count, word)) # добавляем их в список в виде кортежа (значение, ключ)

# Строки 2-3 можно записать так:
# for pair in list_d:                      # для всех котрежей pair = (word, count) в списке
#     list_swop.append((pair[1], pair[0])) # добавляем элементы кортежа в обратном порядке

print(*list_swop, sep='\n')

(1, 'сегодня')
(3, 'он')
(2, 'гуляет')
(1, 'завтра')
(1, 'спит')
(1, 'послезавтра')


Осталось отсортировать полученный список:

In [ ]:
sorted_list = sorted(list_swop)
print(*sorted_list, sep='\n')

(1, 'завтра')
(1, 'послезавтра')
(1, 'сегодня')
(1, 'спит')
(2, 'гуляет')
(3, 'он')


Но нам нужно по убыванию:

In [ ]:
sorted_list = sorted(list_swop, reverse=True)
print(*sorted_list, sep='\n')

(3, 'он')
(2, 'гуляет')
(1, 'спит')
(1, 'сегодня')
(1, 'послезавтра')
(1, 'завтра')


Возможно, функция сортировки по значению пригодится не только в данной задаче, но и при работе с другими словарями. Поэтому переменную `word` переименуем в `key` (_ключ_), а переменную `count` — в `value` (_значение_). И дадим пользователю возможность выбирать, по возрастанию или по убыванию значений сортировать словарь. Итого, получаем следующую функцию:

In [ ]:
# Функция сортировки словаря по значениям
# Входные параметры: словарь, True или False для сортировки по убыванию и возрастанию соответственно
# Возвращаемые значения: отсортированный по значениям список кортежей (значение, ключ)

def sort_dict_by_value(dictionary, reversed_order):
    list_d = list(dictionary.items())   # Преобразуем словарь в список
    list_swop = []                      # Создаем пустой список, в котором ключ и значение поменяются местами
    for key, value in list_d:           # Последовательно для всех ключей и значений:  
        list_swop.append((value, key))  # добавляем их в список в виде кортежа (значение, ключ)
    list_sorted = sorted(list_swop, reverse=reversed_order) # Сортируем список кортежей по убыванию
    return list_sorted                  # Возвращаем отсортированный список кортежей

Запустим ее на большом словаре слов:

In [ ]:
sorted_list = sort_dict_by_value(dict_of_words, True) # Получаем из словаря отсортированный по убыванию значений список
print(*sorted_list[:10], sep='\n')                    # Печатаем 10 самых популярных слов

(1176, 'и')
(733, 'я')
(691, 'в')
(584, 'не')
(446, 'что')
(424, 'с')
(422, 'на')
(291, 'он')
(291, 'меня')
(258, 'мне')


Видим, что по наиболее часто употребляемым словам нельзя ничего узнать о тексте. Требуется дополнительная обработка.

# 4. Удаление стоп-слов

Если мы хотим получить какую-то информацию о тексте по наиболее часто употребяемым словам, необходимо очистить текст от так называемых стоп-слов: в основном это местоимения и служебные части речи, которых много в каждом тексте и которые не несут информации о содержании текста.

Создадим список с общепринятыми стоп-словами для русского языка (список заимствован из модуля для обработки естественного языка Natural Language Toolkit и дополнены несколькими словами).

In [ ]:
stopwords = ['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между', 'сказал','это','сказала']

Создаем новый список всех слов текста, в который войдут только слова, не являющиеся стоп-словами (то есть не входящие в список стоп-слов).

In [ ]:
words_without_stopwords = []                 # Создаем пустой список
for word in words:                           # Последовательно для всех слов из списка:
    if word not in stopwords:                # если слово не находится в списке стоп-слов,
        words_without_stopwords.append(word) # добавляем его к новому списку
print(words_without_stopwords[25:35])        # Смотрим фрагмент получившегося списка, где видно, что стоп-слова исключены

['отец', 'андрей', 'петрович', 'гринев', 'молодости', 'своей', 'служил', 'графе', 'минихе', 'вышел']


Создадим словарь по списку без стоп-слов и распечатаем несколько ключей и значений.

In [ ]:
dict_without_stopwords = create_dict(words_without_stopwords)
print_dict(dict_without_stopwords, 25, 35) 
# Слова "отец" и "андрей" в этом диапазоне не вывелись, 
# так как слово "отец" употреблялось в тексте ранее 
# (и слово "гвардии" встречалось дважды)

петрович 6
гринев 3
молодости 2
своей 17
служил 2
графе 1
минихе 1
вышел 15
отставку 1
премьер-майором 1


Осталось отсортировать список — используем подготовленную функцию.

In [ ]:
sorted_without_stopwords = sort_dict_by_value(dict_without_stopwords, True)
print(*sorted_without_stopwords[:10], sep='\n') # Печатаем 10 самых частых слов

(88, 'пугачев')
(85, 'отвечал')
(75, 'марья')
(75, 'ивановна')
(74, 'тебе')
(66, 'савельич')
(62, 'иван')
(61, 'батюшка')
(55, 'швабрин')
(55, 'крепости')


Теперь наиболее часто встречаемые слова текста можно назвать его _ключевыми словами_ — по ним можно узнать, как минимум, нескольких главных героев произведения.

# 5. Запись в файл

Когда все операции выполнены, может возникнуть потребность сохранить результаты в файл. Реализуем ситуацию, когда нужно сохранить список без стоп-слов по убыванию частоты встречаемости в тексте.

In [ ]:
with open('words.txt', 'w') as outfile:                      # Выбираем имя файла, указываем, что это файл для записи: 
                                                             # 'w' (write)
    print(*sorted_without_stopwords, sep='\n', file=outfile) # Печатаем отсортированный список файл 

Сейчас в файле есть скобки:

In [ ]:
print(*sorted_without_stopwords[:7], sep='\n')

(88, 'пугачев')
(85, 'отвечал')
(75, 'марья')
(75, 'ивановна')
(74, 'тебе')
(66, 'савельич')
(62, 'иван')


Сделаем по-другому: будем записывать в файл в цикле, заодно поменяем местами слово и частоту.

In [ ]:
with open('words.csv', 'w') as outfile:   # Можно указать и 'a' (append) — если файла нет или он пустой и 
                                          # мы его открываем всего 1 раз, то 'a' сработает так же, как и 'w'
    for word in sorted_without_stopwords: # Последовательно для всех слов в отсортированном списке без стоп-слов
        print(word[1], word[0], sep=', ', file=outfile) # На нулевой позиции кортежа лежит частота, а на первой — слово

Полученный файл можем открыть в виде таблицы, например, в Google Sheets.

# Задание для любознательных 
_не проверяется, не оценивается_

Как вы могли заметить, в результате сортировки слова с одинаковой частотой выстроились в обратном порядке: от Я до А. Особенно это заметно, если смотреть в конец списка, где количество употреблений каждого слова равно 1. Но вся сортировка делалась для поиска самых популярных слов, у которых редко совпадают частоты. То есть конец списка — это "побочный продукт". Тем не менее, может возникнуть желание исправить эту ситуацию. **Подумайте, как можно отсортировать так, чтобы слова шли по убыванию количества употреблений, а слова с одинаковым количеством употреблений — по возрастанию (в алфавитном порядке).**

Один из вариантов решения представлен ниже. Сначала попробуйте придумать свой способ, а потом — разобраться в предложенном решении. Раскомментируйте код, чтобы запустить программу.

In [ ]:
# # Определяем функцию для правильной сортировки: частоты – по убыванию, слова с одинаковой частотой — от А до Я
# # Входные параметры: список кортежей (частота, слово), отсортированный по убыванию частот
# # Возвращаемые значения: такой же список кортежей, но слова с одинаковой частотой — в алфавитном порядке

# def new_sort(old_list):
#     counts = []                 # Создаем пустой список для записи частот
#     for word in old_list:       # Последовательно для всех слов в исходном списке
#         counts.append(word[0])  # добавляем частоты в список частот
#     counts_set = set(counts)    # Создаем множество уникальных частот
#     counts = list (counts_set)  # Преобразуем его в список
#     counts.sort(reverse = True) # Сортируем по убыванию
# #    print('Частоты:', counts)  # Смотрим, что получилось
    
#     new_list = []               # Создаем новый список, в котором будет правильная сортировка
#     i = 0                       # Заводим счетчик
#     for count in counts:        # Последовательно для всех частот из списка частот
#         sublist = []            # создаем подсписок, в котором будут только слова с данной частотой
#         while i < len(old_list) and old_list[i][0] == count: # пока не вышли за пределы списка слов и частота
#                                                              # данного слова равна текущей частоте,
#             sublist.append(old_list[i]) # добавляем слово в подсписок
#             i += 1                      # Увеличиваем счетчик
#         new_list += sorted(sublist)     # Добавляем отсортированный подсписок к новому списку
#     return new_list                     # Возвращаем новый список

In [ ]:
# new_list = new_sort(sorted_without_stopwords) 
# print(*new_list[8400:], sep='\n')